# SMRT tutorial for ECCC


***
##### by Julien Meloche 



# Snow Microwave Radiative Tranfer (SMRT) model





Amazing work from Ghislain Picard, Melody Sandells and Henning Löwe!

- Build on previous knowledge on radiative transfer modelling (DMRT and MEMLS)
- Allows for intercomparison of different model elements: electromagnetic theory, microstructure, ice permittivity and much more!
- Modularity (we keep adding more stuff!)
- Written in Python

At ECCC

We will mainly use it to estimate backscatter from snow properties.

## Simply put
### SMRT allows you to simulate measurements of radar or radiometer from snow and or ice properties.

<img src = "SWE_retrieval_SMRT.png" width =900>

***
<br>

#### Radar and Snow!

$ \sigma_{tot}^{pq} =  \sigma_{vol}^{pq} + \sigma_{bg}^{pq} +\sigma_{interface}^{pq}$

for a particular polarisation state (pq)

<img src="backscatter_interaction_Tsangetal2022.png">

<br><br>
Tsang et al 2022 doi :  https://doi.org/10.5194/tc-16-3531-2022


- Snow generates volume scattering from snow grains. Snow is represented as a medium of ice/air (microstructure).
- The background also generates a strong return in backscatter. We need to account for it by estimating the soil permittivity and roughness.
- Backscatter from the reflection of interface is minimal. It is the strongest at the air-snow layer because of the permittivity change ($\varepsilon_{snow} \approx  1.6$)

<br>
wise word for Ghislain Picard (SMRT git hub page)

- Backscatter of a snowpack with flat surface is only due to the grains/ice nodules (assuming no underlying soil or rough ice). A rough surface has usually little impact for dry snowpack because the permittivity of light snow is not large (~1.6).
- Adding water near the surface increases (a lot) the absorption, so the wave penetrates less -> less scattering -> less backscatter. If snow get wet, not only the absorption increases, but also the refractive index (see Fig 1 https://doi.org/10.5194/tc-2022-85). Backscatter from the surface increases as a function of the liquid water content of the superficial layers (say a few centimeters). This is what is observed for soil.

***
#### Polarisations

for a particular polarisation state (pq) (e.g $\sigma_{vol}^{VH}$, emmited = vertical, received = horizontal)

##### Co-polarisation
- VV : vertical polarisations - sensitive to vertical structure or bulk properties of the snowpack
- HH : Horizontal polarisations - sensitive to horizontal structure like ice lense and layering.

##### Cross-polarisation (X-pol)
- VH or HV: when emmitted differs from the received polarisations - sensitive to depolarisation of the waves (volume scattering and snow anisotropy)

#### TSMM will use VV and VH pol

***

### SMRT use the DORT solver yo solve the radiative transfer equation (RTE) through all layers

#### RTE:
$ \large \mu \frac{\partial \mathbf{I}(\mu,\phi,z)}{\partial_z} = -\kappa_e(\mu,\phi,z)\mathbf{I}(\mu,\phi,z)+ \frac{1}{4\pi} \iint_{4\pi}P(\mu,\phi;\mu',\phi',z)\mathbf{I}(\mu',\phi',z)d\Omega'+\kappa_a(\mu,\phi,z)\alpha T(z)1$
<br><br>
<br>
<img src="Multilayered-medium_SMRT.jpg">
<br>

### Final output is $T_B$ or $\sigma^0$ with polarisation H and V
***

### Snowpack Data or inputs

- Multi layered snowpack (density, temperature, ...)


<img src="SMRT.png" width = 700>


### Electromagnetic model
#### Yields ka, ks and effective permittivity for EM theory and microstructure model.


- Dense Media Radiative Transfer (DMRT) : for example, using DMRT, represents the microstructure as aggregates of ice spheres in air are represented in each snow layers using the equivalent diameter = $D_{0}$  and stickiness ($\tau$).
<br><br>
$ \large D_{0} = \frac{6}{SSA \rho_i} $
<br><br>
- Improved Born Approximation (IBA) : represents snow layer in a random two phase ice/air medium. 
- ACF of the medium can be assumed to be exponential but you don't have to! Numerous microstructure (or ACF) can be used with IBA, even dense media. 
- One easy way would be to define the correlation length (porod length) of the snow microstructure with the debye equation using the snow density $\rho_s$ and the SSA.
<br><br>
$ \large l_c = \kappa \frac{4(1-\phi_i)}{\rho_i SSA} $
<br><br>
where the volume fraction is define as such $ \phi_i = \rho_s/\rho_i$.
<br><br>
Sometimes, an empirical factor ($\kappa$) called the debye factor is used to adjust the scattering depending on frequency and region but we should avoid this in the future!!
<br><br>
check this great article https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2021AV000630

#### Polydispersity 
We redefine $l_c$ --> $l_{MW}$  <br><br>
$l_{MW}$ = $K_{medium}$ × $l_{(p,medium)}$ <br><br>

The goal would be to provide polydispersity from grain type

***

For this tutorial <br>
We will use IBA with exponential microstructure and polydispersity = 1

***

#### Even with all this complexity, SMRT can be easy and simple to use.

In [ ]:
from smrt import make_snowpack, make_model, sensor_list
from smrt.utils import dB

#prepare inputs in SI
thick = [100] #m
corr_length = [8e-5] # 80 microns
temperature = [270] # kelvin
density = [320] #kg/m^3

#create the snowpack
snowpack = make_snowpack(thickness = thick,
                         microstructure_model = "exponential",
                         density = density,
                         temperature = temperature,
                         corr_length = corr_length)

#create the sensor
#set frequency to 17.25 GHz and incident angle to 35
radar = sensor_list.active(17.25e9, 35)

#create the model
m = make_model("iba", "dort")

#run the model
result = m.run(radar, snowpack)

#outputs
print(f' sig VV = {result.sigmaVV()}, sig HH = {result.sigmaHH()}')
print(f' sig VV = {dB(result.sigmaVV())}, sig HH = {dB(result.sigmaHH())}')

### SMRT also allows you to set a substrate and atmosphere along with your snowpack

<img src="atmo_substrat_smrt.png" width="400">

### Other cool stuff
- liquid water
- saline snow
- sea ice (FYI and MYI)
- rough surface and interface of layers
- new sensor -> Radar altimetery which adds a time dependent RTE solver

### More info

- Documentation : 
https://smrt-model.science/documentation.html

- API Documentation :
https://smrt.readthedocs.io/en/master/index.html

- Github page : 
https://github.com/smrt-model

- Reference paper : 
https://gmd.copernicus.org/articles/11/2763/2018/gmd-11-2763-2018.html

***

# How to install SMRT

### Install Python and dependencies
#### Easiest with Anaconda
SMRT is developed with the highest stable version of Python but also work with python 2.7 series and 3.4 or higher. Anaconda is the recommended distribution to get Python as it contains numerous packages needed for scientific computations and analysis. This is an open source distribution available for Linux, Windows and MacOS.

In the anaconda prompt ... <br><br>
create an virtual environment for SMRT
```console
conda create --name smrt
```
then activate your env
```console
conda activate smrt
```
install dependencies
```console
conda install numpy scipy xarray pandas six 
```

### You have two option to install SMRT...
#### Not recommended because not up to date. SMRT is in constant developpement.
#### 1. Using pip install from pypi.org:  https://pypi.org/project/smrt/#description

install pip in your env
```console
conda install pip
```
install smrt with pip
```console
pip install smrt
```

#### 2. Best way is to clone the SMRT repo from github.
Make sure git is install on your computer (usually already install on ubuntu)

If not, I like git bash on windows; check to install on your machine https://git-scm.com/download/win

Open a git bash terminal, run this command where you want smrt to clone the repo.
##### Github page : https://github.com/smrt-model

```console
git clone https://github.com/smrt-model/smrt.git
```
##### Optional : Clone smrt tutorials for some exercices!
```console
git clone https://github.com/smrt-model/tutorials.git
```

Check the directory of SMRT
```console
cd smrt
pwd
```
You can go back to this directory to make sure you are up to date new changes in SMRT. <br>
Run this command
```console
git pull
```

Now you add to specify to python the path to smrt
##### Linux
add this to your ~/.bashrc file
```console
PYTHONPATH=$PYTHONPATH:/the/full/path/to/smrt
```

##### Windows
1. Go to settings, search for 'edit the system environment variables'
2. Click on 'Environment Variables'
3. Create new system variable
4. Name it PYTHONPATH
5. Enter or browse to the path to your SMRT directory


## Try to import SMRT in python now!!
***

### First, lets simulate backscatter and brigthness temperature for different angles

In [ ]:
#import library we need for processing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#import smrt function that are needed
from smrt import make_model, make_snowpack, make_soil, sensor_list
from smrt.utils import dB

In [ ]:
#prepare inputs in SI
thick = [0.5, 1] 
corr_length = [2e-4, 5e-5]
temperature = [270, 268] 
density = [280, 400] #kg/m^3

#add frozen soil
soil = make_soil('geometrical_optics', complex(2,0.005), temperature = 265, mean_square_slope = 0.011)

#create the snowpack
snowpack = make_snowpack(thickness = thick,
                         microstructure_model = "exponential",
                         density = density,
                         temperature = temperature,
                         corr_length = corr_length,
                         substrate = soil)
snowpack

In [ ]:
# prepare a list for theta from  10 to 70 by step of 5
theta = np.arange(10,70,5)

# prepare one sensor at 17.5 GHz
radar = sensor_list.active(17.5e9, theta)


# prepare the model and run it successively for each sensor
m = make_model("iba", "dort")
res = m.run(radar, snowpack)

In [ ]:
# results are xarray!
res.sigmaVV()

In [ ]:
#result has a dimension theta because we provided a list of of angles
res.sigmaVV()

In [ ]:
#Same for passive
#add frozen soil
soil = make_soil('soil_wegmuller', complex(3,0.005), temperature = 265, roughness_rms = 0.017)

#create the snowpack
snowpack = make_snowpack(thickness = thick,
                         microstructure_model = "exponential",
                         density = density,
                         temperature = temperature,
                         corr_length = corr_length,
                         substrate = soil)

# prepare one sensor at 37 GHz
radiometer = sensor_list.passive(37e9, theta)

# prepare the model and run it successively for each sensor
m = make_model("iba", "dort")
res_p = m.run(radiometer, snowpack)

res_p.TbV()

In [ ]:
#lets do a plot
fig, (ax1, ax2) = plt.subplots(1, 2,figsize = (12,4))
plt.rcParams.update({'font.size': 14})
ax1.ticklabel_format(axis='both', style='sci', scilimits=(0,2))
ax1.plot(theta, dB(res.sigmaVV()), 'k-',label = 'VV')
ax1.plot(theta, dB(res.sigmaHH()), 'r:',label = 'HH')
ax1.set_ylabel('$\sigma^0 (dB)$')
ax1.set_xlabel("$\\theta$")
ax1.legend(loc = 3)


ax2.plot(theta, res_p.TbV(), label = 'TbV')
ax2.plot(theta, res_p.TbH(), label = 'TbH')
ax2.set_ylabel('$T_B$ (K)')
ax2.set_xlabel('$\\theta$')
ax2.legend()

***
### Sensitivity study of snow parameters on $\sigma^0$

##### Reminder: Equation for SSA
#### Porod length from debye equation
$ \large l_c = \kappa \frac{4(1-\phi_i)}{\rho_i SSA} $
<br><br>
where the volume fraction is define as such $ \phi_i = \rho_s/\rho_i$.

In [ ]:
from smrt.core.globalconstants import DENSITY_OF_ICE

def ssa_to_l(ssa, density):
    #converting SSA to correlation length
    f = density/DENSITY_OF_ICE
    l_d = 4 * (1-f)/(DENSITY_OF_ICE*ssa)
    return l_d

In [ ]:
#set constant parameters for the study
thick = [0.5] 
ssa = 17
temperature = 268
density = 320

#add frozen soil
soil = make_soil('geometrical_optics', complex(2,0.005), temperature = 265, mean_square_slope = 0.011)

#set radiometer for SWE retrievals
radar_ku = sensor_list.active([13.5e9, 17.25e9], 45)
m = make_model("iba", "dort")


#sensitivity of depth
depth_array = np.linspace(10,401,10)/100
snowpack = [make_snowpack(thickness = [thick], microstructure_model = "exponential", density = density,
                          temperature = temperature, corr_length = ssa_to_l(ssa, density), substrate = soil) for thick in depth_array]

res_depth = m.run(radar_ku, snowpack)

#sensitivity of temperature
temp_array = np.linspace(260,280,10)
snowpack = [make_snowpack(thickness = thick, microstructure_model = "exponential", density = density,
                          temperature = [temp], corr_length = ssa_to_l(ssa, density), substrate = soil) for temp in temp_array]

res_temp = m.run(radar_ku, snowpack)

#sensitivity of density
density_array = np.linspace(200,400,10)
snowpack = [make_snowpack(thickness = thick, microstructure_model = "exponential", density = [density],
                          temperature = temperature, corr_length = ssa_to_l(ssa, density), substrate = soil) for density in density_array]

res_density = m.run(radar_ku, snowpack)

#sensitivity of  SSA
ssa_array = np.linspace(5,40,10)
snowpack = [make_snowpack(thickness = thick, microstructure_model = "exponential", density = density,
                          temperature = temperature, corr_length = ssa_to_l(ssa, density), substrate = soil) for ssa in ssa_array]

res_lc = m.run(radar_ku, snowpack)

### Plot the results

In [ ]:
#create a subfigure for each parameter studied
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (12,9), sharey = True)
plt.rcParams.update({'font.size': 14})

#plot snow depth
ax1.plot(depth_array, dB(res_depth.sigmaVV().sel(frequency = 17.25e9)), label = '17.25 GHz')
ax1.plot(depth_array, dB(res_depth.sigmaVV().sel(frequency = 13.5e9)), label = '13.5 GHz')
ax1.set_xlabel('Snow depth (m)')
ax1.set_ylabel('$\sigma^0$ (dB) VV-pol')
ax1.legend()

#plot temperature
ax2.plot(temp_array, dB(res_temp.sigmaVV().sel(frequency = 17.25e9)).values)
ax2.plot(temp_array, dB(res_temp.sigmaVV().sel(frequency = 13.5e9)).values)
ax2.set_xlabel('Snow temperature (K)')

#plot density
ax3.plot(density_array, dB(res_density.sigmaVV().sel(frequency = 17.25e9)))
ax3.plot(density_array, dB(res_density.sigmaVV().sel(frequency = 13.5e9)))
ax3.set_xlabel('Snow density ($kg/m^3$)')
ax3.set_ylabel('$\sigma^0$ (dB) VV-pol')

#plot corr length
ax4.plot(ssa_array, dB(res_lc.sigmaVV().sel(frequency = 17.25e9)).values)
ax4.plot(ssa_array, dB(res_lc.sigmaVV().sel(frequency = 13.5e9)).values)
ax4.set_xlabel('SSA ($kg^2 m^{-1}$)')

plt.subplots_adjust(wspace = 0.1)


#####  SWE can be estimated empirically. However, the empirical relation might not hold for all snow types and conditions. This is where a radiative transfer can help to link backscatter and snow properties!!

### Add dimension to your result

#### Let's define multiple dimensions and use the full power of xarray and pandas

In [ ]:
#define a list of snowpack
depth_array = np.linspace(10,201,10)/100
ssa = 17
temperature = 268
density = 320
soil = make_soil('soil_wegmuller', complex(3,0.005), temperature = 265, roughness_rms = 0.017)
snowpack = [make_snowpack(thickness = [thick], microstructure_model = "exponential", density = density,
                          temperature = temperature, corr_length = ssa_to_l(ssa, density), substrate = soil) for thick in depth_array]

#sensor can have two frequency
#use the common AMSRE satellite
sensor = sensor_list.amsre(['19', '37'])

#define dimension of the list of snowpack
res_depth = m.run(sensor, snowpack, snowpack_dimension = ('depth', depth_array))
res_depth.Tb()

In [ ]:

snowpack = pd.Series([make_snowpack(thickness = [thick], microstructure_model = "exponential", density = density,
                          temperature = temperature, corr_length = ssa_to_l(ssa, density), 
                          substrate = make_soil('soil_wegmuller', complex(3,0.005),
                                                 temperature = 265, roughness_rms = 0.017)) for thick in depth_array], index = depth_array)

#define dimension of the list of snowpack
res_depth = m.run(sensor, snowpack)
res_depth.Tb()

In [ ]:
#result can also be set to a dataframe
res_depth.to_dataframe()

## Optionnal parameters

 rtsolver_options = {'error_handling':'nan'}
 <br><br>

In [ ]:
ssa = 0.1
temperature = 268
density = 320

#add frozen soil
soil = make_soil('geometrical_optics', complex(2,0.005), temperature = 265, mean_square_slope = 0.01)

#set radiometer for SWE retrievals
radar_ku = sensor_list.active([13.5e9, 17.25e9], 45)
#m = make_model("iba", "dort")
m = make_model("iba", "dort", rtsolver_options = {'error_handling':'nan'})


#sensitivity of depth
depth_array = np.linspace(10,401,10)/100
snowpack = [make_snowpack(thickness = [thick], microstructure_model = "exponential", density = density,
                          temperature = temperature, corr_length = ssa_to_l(ssa, density), substrate = soil) for thick in depth_array]


res_temp = m.run(radar_ku, snowpack)

In [ ]:
res_temp.sigmaVV()

### Parallel computation

#### advantage of running a multiple snowpack
The model can be run on a list of snowpacks or even more conveniently on a pandas.Series or pandas.DataFrame including snowpacks. The first advantage is that by setting parallel_computation=True, the Model.run() method performs the simulation in parallel on all the available cores of your machine and even possibly on a high performance cluster using dask. The second advantage is that the returned Result object contains all the simulations and provide an easier way to plot the results or compute statistics.

The job lib library is used to compute in parallel multiple snowpack <br>
*** needs a list of snowpack when calling run

In [ ]:
res_temp = m.run(radar_ku, snowpack, parallel_computation=True)

### Some wise words from Ghislain Picard (developper)
Question from user: <br>
Do you have any plans to convert this code into C++ ?

There is not plan to convert to a different language. SMRT is fast despite being written in Python. The computational bottleneck is often in the diagonalisation of matrices, which is handled very efficiently with underlying LAPACK. A Fortran / C++ code would be marginally faster.

### Exemple with real data (CROCUS)

In [ ]:
import xarray as xr
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
mod = xr.open_dataset('Pow_out_svs2.nc')

df = mod[['SNODEN_ML','SNOMA_ML','TSNOW_ML','SNODOPT_ML','SNODP']].to_dataframe().dropna() 
# SNODEN_ML: Layer density 
# SNOMA_ML: Layer SWE
# TSNOW_ML: Layer temperature
# SNODOPT_ML: Layer optical diameter
# SNODP: depth of the snowpack

##### Recall optical diameter
$ \large D_{0} = \frac{6}{SSA \rho_i} $

In [ ]:
def Do_to_ssa(Do):
    return 6/( Do * 917)

#add colums for SMRT

#thickness
df['thickness'] = df[['SNODEN_ML','SNOMA_ML']].apply(lambda x : x[1] / x[0], axis = 1) 

#SSA
df['ssa'] = df['SNODOPT_ML'].apply(lambda x: Do_to_ssa(x) if x>0 else 0)
#df['corr_length'] = df[['SNODEN_ML','ssa']].apply(lambda x: debye_eqn(x[1], x[0]), axis = 1)

In [ ]:
#filter out low snowdepth and small snow layers
df = df[(df.SNODP > 0.10) & (df.thickness > 0.005)]
df

In [ ]:
#get dates
dates = df.groupby(level = 'time').mean().index.get_level_values(0)
df.loc[dates[0],:]

In [ ]:
def get_snowpack(snow_df):
    #add frozen soil
    soil = make_soil('geometrical_optics', complex(2,0.005), temperature = 265, mean_square_slope = 0.01)

    #Creating the snowpack to simulate with the substrate
    if isinstance(snow_df.thickness, np.floating):
        thickness = [snow_df.thickness]
    else:
        thickness = snow_df.thickness
        
    sp = make_snowpack(thickness=thickness, 
                        microstructure_model='exponential',
                        density= snow_df.SNODEN_ML,
                        temperature= snow_df.TSNOW_ML,
                        corr_length = ssa_to_l(snow_df.ssa, snow_df.SNODEN_ML),
                        substrate = soil)
    
    return sp

In [ ]:
#get dates
dates = df.groupby(level = 'time').mean().index.get_level_values(0)#
#get total SWE per date
SWE = df.groupby(level = 'time').sum().SNOMA_ML
#get total snowdepth per date
depth = df.groupby(level = 'time').mean().SNODP

#create a series of snowpack object
snowpacks = pd.Series([get_snowpack(df.loc[date,:]) for date in dates ], index = dates)

#create a dataframe
meta_df = pd.DataFrame({'depth' :depth, 'SWE': SWE, 'smrt_snow' : snowpacks}, index = dates).dropna()
meta_df

In [ ]:
#Modeling theories to use in SMRT
model = make_model("symsce_torquato21", "dort", rtsolver_options = {'error_handling':'nan'})

sensor  = sensor_list.active(17.5e9, 35)

result = model.run(sensor, meta_df, snowpack_column='smrt_snow', parallel_computation=True)


In [ ]:
meta_df['sigVV'] = dB(result.sigmaVV())
meta_df['sigHH'] = dB(result.sigmaHH())

In [ ]:
meta_df.head()

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (7,5))
ax.plot(meta_df.index, meta_df.sigVV, color = 'dodgerblue',label = 'VV', alpha = 0.5)
ax.plot(meta_df.index, meta_df.sigHH, color = 'blue', label = 'HH', alpha = 0.5)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.legend()
ax.set_ylabel('$\sigma^0$ (dB)')

ax2=ax.twinx()
ax2.plot(meta_df.index, meta_df.SWE, color = 'darkorange', alpha = 0.7)
ax2.set_ylabel('SWE')
ax2.yaxis.label.set_color('darkorange')
ax2.tick_params(axis='y', colors='darkorange')

## (Advanced) Different flavours of SMRT

 The SMRT model was developed in response to increased understanding of the importance of microstructure parameterization in snow, and to enable isolation of individual model components in microwave scattering model intercomparison studies. For this, SMRT has a modular structure, which clearly separates the different steps of the calculation (permittivities of the raw materials, scattering coefficients, and solution of the radiative transfer (RT) equation) (Sandells et al. 2022)

### Electromagnetic Models 
- DMRT (Dense media)
- IBA (Improved Born Approximation)
- symSCE (symmetrized Strong Constrast Expansion)

### Microstrure models (autocorrelation function of ice/air matrix which change calculation of $k_s$)


In [ ]:

#define function to get microstructure parameters from ssa
def ssa_to_R(ssa, density):
    #converting SSA to eq radius
    return 3/(DENSITY_OF_ICE*ssa)


In [ ]:
#start from SSA, which you would get from field measurements
ssa_array = np.arange(10, 65, 5)

#set other parameters for the study
#thick must be a list or array to infer number of layer
thick = [0.5] 
temperature = 268 
density = 320

#add frozen soil
soil = make_soil('soil_wegmuller', complex(3.34,0.005), temperature = 265, roughness_rms = 0.017)

In [ ]:
#create sensor for amsre channel
radio  = sensor_list.amsre('37')

#set electromagnetic model
#iba can run most of the microstructure model
m_iba = make_model("iba", "dort")
#DMRT only can run SHS
m_dmrt = make_model('dmrt_qca_shortrange', 'dort')
#SCE strong contrast expansion (new model for high density)
m_sce = make_model("sce_torquato21", "dort")

#set snowpack with different microstructure model
#Microstructure exponential
snowpack_exp = [make_snowpack(thickness = thick, microstructure_model = "exponential", density = density,
                          temperature = temperature, corr_length = [ssa_to_l(ssa, density)], substrate = soil) for ssa in ssa_array]
#Microstructure IND
snowpack_ind = [make_snowpack(thickness = thick, microstructure_model = "independent_sphere", density = density,
                          temperature = temperature, radius = [ssa_to_R(ssa, density)], substrate = soil) for ssa in ssa_array]
#Microstructure SHS
snowpack_shs = [make_snowpack(thickness = thick, microstructure_model = "sticky_hard_spheres", density = density,
                          temperature = temperature, radius = [ssa_to_R(ssa, density)], stickiness = 0.3, substrate = soil) for ssa in ssa_array]

#run result for iba EXP
res_iba_exp = m_iba.run(radio, snowpack_exp, snowpack_dimension = ('ssa', ssa_array))

#run result for iba IND
res_iba_ind = m_iba.run(radio, snowpack_ind, snowpack_dimension = ('ssa', ssa_array))

#run result for iba SHS
res_iba_shs = m_iba.run(radio, snowpack_shs, snowpack_dimension = ('ssa', ssa_array))

#run result for DMRT SHS
res_dmrt_shs = m_dmrt.run(radio, snowpack_shs, snowpack_dimension = ('ssa', ssa_array))

#run result for sce EXP
res_sce_exp = m_sce.run(radio, snowpack_exp, snowpack_dimension = ('ssa', ssa_array))

In [ ]:
#create a subfigure for each parameter studied
fig = plt.figure(figsize = (8,5))
plt.rcParams.update({'font.size': 14})

#plot the results
plt.plot(res_iba_exp.ssa, res_iba_exp.TbV(), label = 'IBA-EXP')
plt.plot(res_iba_ind.ssa, res_iba_ind.TbV(), label = 'IBA-IND')
plt.plot(res_iba_shs.ssa, res_iba_shs.TbV(), label = 'IBA-SHS')
plt.plot(res_dmrt_shs.ssa, res_dmrt_shs.TbV(), label = 'DMRT-SHS')
plt.plot(res_sce_exp.ssa, res_sce_exp.TbV(), label = 'SCE-EXP')
plt.xlabel('SSA ($m^2/kg)$')
plt.ylabel('$T_B$ (K)')
plt.legend()

#### The choice of the microstructure and electromagnetic model does have an impact, especially for low SSA in this case.
***

## Conclusion


- Allows you to simulate measurements of snow or ice from radiometer and RADAR.
- Comparaison between models components (electromagnetic, microstructure, ...) and unified radiative transfer modelling into one models.
- Easy to use with python!

ECCC focus:
- allows us to estimate SWE from backscatter of dual Ku-band by using snow properties from SVS2



#### For more exercices... see the tutorials on the github page

### More info

- Documentation : 
https://smrt-model.science/documentation.html

- Github page : 
https://github.com/smrt-model

- Reference paper : 
https://gmd.copernicus.org/articles/11/2763/2018/gmd-11-2763-2018.html

***